In [6]:
!pip install rank_bm25

In [7]:
# 필요한 라이브러리 임포트
# pandas: 데이터 처리를 위한 라이브러리
# rank_bm25: BM25 알고리즘을 사용하기 위한 라이브러리
from rank_bm25 import BM25Okapi
import pandas as pd

In [8]:
from google.colab import files
uploaded = files.upload()

Saving sample_qna.csv to sample_qna (1).csv


In [9]:
qna = pd.read_csv('sample_qna.csv', encoding='utf-8', index_col=False)
qna = qna.loc[:22,:]
qna.head(10)

,문의유형,문의내용,답변,비고
0,회원가입하는 방법,회원가입하는 방법 알려주세요. 회원가입 어떻게 하나요?,"회원가입 자격조건은 따로 없으며, 회원가입 버튼을 통해 가입 가능합니다.\n+ 회원...",NaN
1,ID 찾기 문의,아이디 조회가 안돼요. 제 아이디 알려주세요.,아이디 찾기를 통해 조회가 가능합니다. 아이디를 생성하고 2년 안에 개인정보 활용동...,NaN
2,PW 찾기 문의,비밀번호가 기억이 안나요. 비밀번호 알려주세요.,비밀번호 찾기를 통해 비밀번호 재설정이 가능합니다.\n+ 비밀번호찾기 바로가기링크 ...,NaN
3,로그인 오류,"로그인이 안돼요(비밀번호 5회 틀렸어요, 회원가입할때 오류)",확인이 필요한 사항이니 NTIS센터 콜센터(042-869-1115)로 연락을 주시면...,NaN
4,아이디 정보 변경,아이디에 바꾸고싶은 정보가 있어요.,로그인 후 '내정보 - 내정보 관리'에서 수정 가능합니다. \n직접 수정이 어려우신...,"""내정보관리"" 바로가기링크 추가되나요?"
5,아이디 정보 변경(개명),개명했는데 어떻게 바꿔요?,NTIS센터 콜센터(042-869-1115)로 연락주시면 본인확인 후 개명 처리 가...,NaN
6,국가연구자번호 관련,국가연구자번호 관련 모든 문의,국가연구자번호 서비스는 IRIS로 이관되었습니다.\n국가연구자 관련 모든 문의는 I...,NaN
7,회원 탈퇴,회원탈퇴 하고싶어요,로그인 후 '내정보' 클릭 후 우측 상단의 '회원탈퇴' 버튼 클릭을 통하여 회원탈퇴...,"""회원탈퇴"" 바로가기링크 추가되나요?"
8,개인정보활용동의,개인정보활용동의는 어디서 어떻게 하나요?,로그인 후 '내정보 - 개인정보활용동의'를 통하여 가능합니다.,"""개인정보활용동의"" 바로가기링크 추가되나요?"
9,권한신청,권한신청은 어디서 하나요?,"권한신청은 해당자격이 있는 사람만 신청 가능하며, 자격 조건 확인 후 권한 신청하시...","""권한신청"" 바로가기링크 추가되나요?"


In [10]:
# 데이터 전처리 함수 정의
def preprocess_function(examples):
    """
    데이터셋을 받아서 질문과 답변을 매칭하는 딕셔너리로 변환합니다.

    examples: pandas DataFrame - 질문과 답변이 포함된 데이터프레임
    반환값: dict - 질문을 키(key), 답변을 값(value)으로 가지는 딕셔너리
    """
    user = [examples.loc[idx,"문의내용"] for idx in examples.index ]
    responses = [examples.loc[idx,"답변"] for idx in examples.index]

    dict_response={}
    for i in range(len(user)):
        dict_response[user[i]] = responses[i]

    return dict_response

In [11]:
# 간단한 토크나이저 함수 정의 (띄어쓰기 기준으로 단어 분리)
def tokenizer(oneexample):
    """
    텍스트를 공백으로 구분하여 토큰화합니다.

    text: str - 입력 텍스트
    반환값: list - 토큰 리스트
    """
    return (oneexample.split(' ') )

In [12]:
# BM25 모델 생성 함수 정의
def make_index(examples):
    """
    사용자 데이터를 받아 BM25 인덱스를 생성합니다.

    user_data: list - 질문 리스트
    반환값: BM25Okapi - BM25 모델
    """
    # 질문 리스트를 토큰화 (띄어쓰기 기준으로 분리)
    tokenized_user = [examples.loc[idx,"문의내용"].split(' ') for idx in examples.index]
    # BM25 모델 생성
    bm25 = BM25Okapi(tokenized_user)
    return(bm25)

In [13]:
# BM25 모델 생성
bm25 = make_index(qna)

In [14]:
dict_response = preprocess_function(qna)
user_data = [qna.loc[idx,"문의내용"] for idx in qna.index ]

In [15]:
# 챗봇 인터페이스
for i in range(5):  # 5번의 대화를 진행
    query = input(">>User: ")  # 사용자로부터 질문 입력 받기
    tokenized_query = tokenizer(query)  # 입력된 질문을 토큰화
    matched_query = ' '.join(bm25.get_top_n(tokenized_query, user_data, n=1))  # 가장 유사한 질문 찾기
    match_score = max(bm25.get_scores(tokenized_query))  # 매칭 점수 계산
    response = dict_response[matched_query]  # 해당 질문에 대한 답변 찾기
    print(">> Bot: {} ({})".format(response, match_score))  # 답변 출력

>>User: 아이디 몰라요
>> Bot: 아이디 찾기를 통해 조회가 가능합니다. 아이디를 생성하고 2년 안에 개인정보 활용동의를 
안하셨을 경우 파기되었을 가능성이 있습니다. (동일한 아이디로 재가입 불가)
+ 아이디찾기 바로가기링크 추가
아이디 찾기가 어려운 경우 NTIS콜센터(042-869-1115))로 문의주시기 바랍니다. (3.6349667128888723)


KeyboardInterrupt: Interrupted by user